https://docs.verilogtorouting.org/en/latest/vpr/file_formats/#vpr-pack-file 

In [1]:
import xml.etree.ElementTree as ET
import os
import numpy as np
import subprocess
import pickle
from ipyfilechooser import FileChooser
import glob

In [2]:
packed_netlists = glob.glob('/home/marieke/Documents/HPC_output/output/*/*r*q*s*.net')
len(packed_netlists)

25

In [3]:
bm_folder = '../benchmarks/'
hmetis = '../hmetis-1.5-linux/hmetis'

output_folder = os.path.join(bm_folder, 'rent_files')

packed_netlist = packed_netlists[0]
file_name = os.path.split(packed_netlist)[-1]
path_graphfiles_folder = os.path.join(bm_folder,'graphfile' ,file_name)
if not os.path.isdir(path_graphfiles_folder):
    os.mkdir(path_graphfiles_folder)



In [9]:
class Hypergraph:
    def __init__(self, hypergraph, node_names, node_sizes, external_edges, folder, name='hg', suffix=''):
        # Extract hypergraph nodename = list index + 1 (starts from 1), list of edgenames per node
        self.hypergraph = np.array(hypergraph, dtype=object)
        
        self.node_names = np.array(node_names)
        self.node_sizes = np.array(node_sizes)
        # Input and output edges are external edges
        self.external_edges = np.array(external_edges)

        # Exclude external edges that are not connected to a node in the hypergraph
        external_edges_mask = np.zeros(len(self.external_edges))
        for edges in hypergraph:
            for edge in edges:
                indices = np.argwhere(self.external_edges == edge)
                if indices.size > 0:
                    index = np.argwhere(self.external_edges == edge)[0]
                    external_edges_mask[index] = 1

        self.external_edges = self.external_edges[external_edges_mask == 1]

        self.n_vertices = len(self.hypergraph)  ## Blocks
        self.n_pins = len(self.external_edges)  ## Pins

        self.folder = folder
        self.name_base = name
        self.suffix = suffix
    def get_n_blocks(self):
        return sum(self.node_sizes) 
    def get_n_pins(self):
        return self.n_pins
    
    def print_hmetis_hypergraph(self):
        # Hmetis expects flipped graph (edge -> nodes) instead of (node -> edges)
        edge_nodes = {}
        for i, edges in enumerate(self.hypergraph):
            node = str(i + 1)
            for edge in edges:
                if edge in edge_nodes:
                    edge_nodes[edge].append(node)
                else:
                    edge_nodes[edge] = [node]
        self.n_hyperedges = len(edge_nodes)
        self.n_vertices = len(self.hypergraph)
        # Print file
        hmetis_lines = []
        hmetis_lines.append(' '.join([str(self.n_hyperedges), str(self.n_vertices), '10']))
        for nodes in edge_nodes.values():
            hmetis_lines.append(' '.join(nodes))
        for weight in self.node_sizes:
            hmetis_lines.append(str(weight))
        file1 = open(self.get_path_graphfile(), 'w')
        lines = file1.writelines([entry + '\n' for entry in hmetis_lines])
        file1.close()

    def run_hmetis(self, hmetis_path):
        #print(hmetis_path, self.get_path_graphfile())
        output = subprocess.run([hmetis_path, self.get_path_graphfile(), '2', '5', '10', '1', '1', '2', '1', '0'],
                                capture_output=True)
#         print(output)
        
    def split(self, hmetis_path):
        # make input file for hmetis
        self.print_hmetis_hypergraph()
        # run hmetis
        self.run_hmetis(hmetis_path)
        # process output file (name inputfile + '.part.2') --> format read hmetis docs
        path_splitfile = self.get_path_graphfile() + '.part.2'
        file1 = open(path_splitfile, 'r')
        lines = file1.readlines()
        file1.close()

        # split hypergraph nodes
        mask = np.array(list(map(int, lines)))  # partition1: 0, partition2: 1
        hypergraph0 = self.hypergraph[mask == 0]
        hypergraph1 = self.hypergraph[mask == 1]

        # add cut edges to external edges
        cut_edges = np.array(list(set(np.concatenate(hypergraph0)) & set(np.concatenate(hypergraph1))))
        self.external_edges = np.unique(np.append(self.external_edges, cut_edges))

        hg0 = Hypergraph(self.hypergraph[mask == 0], self.node_names[mask == 0], self.node_sizes[mask == 0], self.external_edges, self.folder, self.name_base, '0')
        hg1 = Hypergraph(self.hypergraph[mask == 1], self.node_names[mask == 1], self.node_sizes[mask == 1], self.external_edges, self.folder, self.name_base, '1')

        return hg0, hg1

    def get_path_graphfile(self):
        return os.path.join(self.folder, self.name_base + self.suffix)

class subckt:
    def __init__(self, name, inputs, outputs):
        self.inputs = inputs
        self.outputs = outputs
        self.name = name

In [10]:
def get_hypergraph_from_net(root, path_graphfiles_folder):

    hypergraph_internal = []
    node_names = []
    node_sizes = []
    for xml_block in root.findall('block'):
        if xml_block.attrib['mode'] != 'io':
            edges = []
            size = 0
            instance = xml_block.attrib['instance'].split('[')[0]
            if instance == 'LAB' :
                size=1
            elif instance == 'M9K':
                size=1
            elif instance == 'DSP':
                size=4
            inputs = xml_block.find('inputs')
            outputs = xml_block.find('outputs')
            for port in inputs.getchildren():
                edges.extend(port.text.split())
            for port in outputs.getchildren():
                edges.extend(port.text.split())      

            hypergraph_internal.append([net for net in edges if net !='open'])
            node_names.append(xml_block.attrib['name'])
            node_sizes.append(size)
        
    inputs = root.find('inputs').text.split()
    outputs = root.find('outputs').text.split()

    # Input and output edges are external edges
    external_edges = []
    external_edges.extend(inputs)
    external_edges.extend(outputs)
    hypergraph = Hypergraph(hypergraph_internal, node_names, node_sizes, external_edges, path_graphfiles_folder)
    return hypergraph

In [11]:
def bipartition(hg, rent_data, hmetis_path,  partition_level=0):
    vertices = hg.n_vertices
    blocks = hg.get_n_blocks()
    pins = hg.get_n_pins()
    if len(rent_data) >= partition_level + 1:
        rent_data[partition_level].append([blocks, pins])
    else:
        rent_data.append([[blocks, pins]])
    if vertices > 2:
        hg0, hg1 = hg.split(hmetis_path)
        del hg
        bipartition(hg0, rent_data, hmetis_path, partition_level+1)
        bipartition(hg1, rent_data, hmetis_path, partition_level+1)

In [12]:
for packed_netlist in packed_netlists:
    print(packed_netlist)
    tree = ET.parse(packed_netlist)
    root = tree.getroot()

    hg = get_hypergraph_from_net(root, path_graphfiles_folder)
    rent_data = []
    bipartition(hg, rent_data,hmetis)

    print(os.path.join(output_folder, os.path.split(os.path.split(packed_netlist)[-1])[-1]+'.rent'))
    with open(os.path.join(output_folder, os.path.split(packed_netlist)[-1]+'.rent'), "wb") as fp:  # Pickling
        pickle.dump(rent_data, fp)

/home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_r0.50_q0.5_s1.00.hnl/sparcT1_core_stratixiv_arch_timing_gnl_r0.50_q0.5_s1.00.hnl.net


/tmp/ipykernel_64906/1586689153.py:19: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  for port in inputs.getchildren():
/tmp/ipykernel_64906/1586689153.py:21: DeprecationWarning: This method will be removed in future versions.  Use 'list(elem)' or iteration over elem instead.
  for port in outputs.getchildren():


../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_r0.50_q0.5_s1.00.hnl.net.rent
/home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_r0.50_q0.5_s0.50.hnl/sparcT1_core_stratixiv_arch_timing_gnl_r0.50_q0.5_s0.50.hnl.net
../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_r0.50_q0.5_s0.50.hnl.net.rent
/home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_r0.45_q0.5_s0.71.hnl/sparcT1_core_stratixiv_arch_timing_gnl_r0.45_q0.5_s0.71.hnl.net
../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_r0.45_q0.5_s0.71.hnl.net.rent
/home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_r0.60_q0.5_s2.00.hnl/sparcT1_core_stratixiv_arch_timing_gnl_r0.60_q0.5_s2.00.hnl.net
../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_r0.60_q0.5_s2.00.hnl.net.rent
/home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_r0.40_q0.5_s1.41.hnl/sparcT1_core_stratixiv_a

In [29]:
rent_data

[[[3406, 194]],
 [[1874, 2773], [1532, 2686]],
 [[1014, 2731], [860, 2640], [823, 1637], [709, 2496]],
 [[558, 2145],
  [456, 1596],
  [386, 2179],
  [474, 1828],
  [437, 998],
  [386, 1054],
  [342, 1761],
  [367, 1819]],
 [[251, 1941],
  [307, 1664],
  [208, 1259],
  [248, 1481],
  [213, 1125],
  [173, 1571],
  [214, 1604],
  [260, 1430],
  [241, 1211],
  [196, 1029],
  [205, 893],
  [181, 948],
  [170, 819],
  [172, 1362],
  [201, 1231],
  [166, 1277]],
 [[112, 1104],
  [139, 1245],
  [142, 1343],
  [165, 1168],
  [96, 780],
  [112, 1036],
  [137, 1094],
  [111, 1055],
  [115, 982],
  [98, 749],
  [79, 826],
  [94, 966],
  [116, 1133],
  [98, 998],
  [122, 917],
  [138, 1125],
  [125, 999],
  [116, 766],
  [108, 849],
  [88, 768],
  [92, 857],
  [113, 644],
  [91, 715],
  [90, 635],
  [85, 560],
  [85, 448],
  [94, 1078],
  [78, 716],
  [90, 955],
  [111, 841],
  [92, 866],
  [74, 804]],
 [[50, 445],
  [62, 886],
  [77, 736],
  [62, 778],
  [79, 821],
  [63, 884],
  [74, 768],
  [91

In [30]:
251+307

558

In [1]:
1/0.8


1.25

import glob
import subprocess

# absolute path to search all text files inside a specific folder
path = r'/home/marieke/Documents/HPC_output/output/*/*cleaned.echo.blif'
files = glob.glob(path)
files

run ../CalculateRent/main.py

for file in files:
    output_folder = os.path.join("../benchmarks/rent_files/", file.split('/')[-2])
    subprocess.run(["mkdir", output_folder])
    os.system(' '.join(["../CalculateRent/main.py", file, output_folder, "../hmetis-1.5-linux/hmetis"]))
    

' '.join(["../CalculateRent/main.py", file, output_folder, "../hmetis-1.5-linux/hmetis"])

run ../CalculateRent/main.py /home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_0.60/atom_netlist.cleaned.echo.blif ../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_0.60 ../hmetis-1.5-linux/hmetis

run ../CalculateRent/main.py /home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_0.50/atom_netlist.cleaned.echo.blif ../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_0.50 ../hmetis-1.5-linux/hmetis

run ../CalculateRent/main.py /home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_0.45/atom_netlist.cleaned.echo.blif ../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_0.45 ../hmetis-1.5-linux/hmetis

../CalculateRent/main.py /home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing_gnl_0.40/atom_netlist.cleaned.echo.blif ../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing_gnl_0.40 ../hmetis-1.5-linux/hmetis

../CalculateRent/main.py /home/marieke/Documents/HPC_output/output/sparcT1_core_stratixiv_arch_timing/atom_netlist.cleaned.echo.blif ../benchmarks/rent_files/sparcT1_core_stratixiv_arch_timing ../hmetis-1.5-linux/hmetis